<a href="https://colab.research.google.com/github/vinothkumard2006-ai/data-science/blob/main/Political_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.listdir('/content/drive/MyDrive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['Untitled spreadsheet.gsheet',
 'job survey.gsheet',
 'final_combined_enriched_v4.csv',
 'Colab Notebooks']

In [27]:
Data = pd.read_csv('/content/drive/MyDrive/final_combined_enriched_v4.csv')

In [28]:
Data.head()

,tweet,type
0,This cartoon by Alok @caricatured speak about...,1
1,I am going to post one #BernieAtTheFarmersProt...,1
2,@sardesairajdeep @OfficialUrmila @RahulGandhi ...,1
3,Oooh... that`s right by the zoo... think... i...,0
4,@vivekoberoi @narendramodi @OmungKumar @suresh...,1


In [29]:
Data.tail()

,tweet,type
50164,ROYALICA Women Black Georgette Anarkali Kurta ...,0
50165,#Latamangeshkar is one of the star campaigner...,1
50166,Om shakthi-sadashiv Om sri-vishnu Jai ho modi...,1
50167,@RahulGandhi @RahulGandhi Shame on you and yo...,1
50168,it can happen only in #india,0


In [30]:
Data.shape

(50169, 2)

In [31]:
Data["type"].value_counts()

,count
type,
1,27459
0,22710


## Check missing values

In [32]:
Data.isna().sum()

,0
tweet,0
type,0


## Very important (Text cleaning)

In [33]:
## WE clean
#1. lowercase
#2. remove URls
#3. remove mentions & hastags
#4. remove punctuation
#5. remove stopwords

In [34]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words=set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'https\S+', '', text)   ## remove urls
    text = re.sub(r'@\w+', '', text)  ## remove mentions
    text = re.sub(r'#\w+', '', text)  ## remove hastags
    text = re.sub(r'[^a-z\s]', '', text)  ## remove punctuations

    text = " ".join([word for word in text.split() if word not in stop_words])

    return text

Data['clean_tweet']=Data['tweet'].apply(clean_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## step 4   Train test split

In [35]:
from sklearn.model_selection import train_test_split

X=Data['clean_tweet']
y=Data['type']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

## step 5 Convert text to numbers(TF-IDF)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(
             max_features=5000,
             ngram_range=(1,2))
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)


## step 6 Train Multiple Models

In [37]:
## 1. Logistic Regression

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train_vec, y_train)

LogisticRegression()

In [38]:
## 2. Navie Bayes

from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train_vec, y_train)

MultinomialNB()

In [39]:
## 3. Support Vector Machine(SVM)

from sklearn.svm import LinearSVC
svm = LinearSVC()
svm.fit(X_train_vec, y_train)

LinearSVC()

## Step 7 Evaluate Models

In [40]:
from sklearn.metrics import accuracy_score,classification_report

def evaluate(model):
    y_pred = model.predict(X_test_vec)
    print("Accuracy:",accuracy_score(y_test,y_pred))


    print(classification_report(y_test,y_pred))
print("Logistic Regression")
evaluate(lr)

print("Navie Bayes")
evaluate(nb)

print("Support Vector Machine")
evaluate(svm)

Logistic Regression
Accuracy: 0.9105042854295395
              precision    recall  f1-score   support

           0       0.88      0.93      0.90      4542
           1       0.94      0.89      0.92      5492

    accuracy                           0.91     10034
   macro avg       0.91      0.91      0.91     10034
weighted avg       0.91      0.91      0.91     10034

Navie Bayes
Accuracy: 0.8690452461630457
              precision    recall  f1-score   support

           0       0.92      0.78      0.84      4542
           1       0.84      0.94      0.89      5492

    accuracy                           0.87     10034
   macro avg       0.88      0.86      0.87     10034
weighted avg       0.87      0.87      0.87     10034

Support Vector Machine
Accuracy: 0.9073151285628862
              precision    recall  f1-score   support

           0       0.88      0.92      0.90      4542
           1       0.93      0.90      0.91      5492

    accuracy                           0

##  Select Highest accuray and f1 score

##  Step 8 Hyperparameter Tuning

In [41]:
from sklearn.model_selection import GridSearchCV

params = {'C':[0.1,1,10]}

grid = GridSearchCV(
    LogisticRegression(),
    params,
    cv=5,
    scoring="f1")

grid.fit(X_train_vec,y_train)
grid.best_params_

{'C': 1}

In [42]:
## Train best model

best_model = grid.best_estimator_

## Step 9 Save Model

In [43]:
import joblib
joblib.dump(best_model,"political_model.pkl")
joblib.dump(tfidf,"tfidf.pkl")

['tfidf.pkl']

## Step 10  Predict New sentence

In [44]:
model = joblib.load("political_model.pkl")
vectorizer = joblib.load("tfidf.pkl")

def predict_sentence(text):
    text = clean_text(text)
    vec = vectorizer.transform([text])
    result = model.predict(vec)[0]

    return "Political"  if result == 1 else "Non-Political"

predict_sentence("The decision sparked debates across universities and workplaces") ## model is successfully  predicted !!!!!

'Non-Political'

# Deep Learning

### Train test split

In [45]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla T4


In [46]:
train_texts, test_texts, train_labels, test_labels=train_test_split(
    Data['tweet'].tolist(),
    Data['type'].tolist(),
    test_size=0.2,
    random_state=42)

## Load BERT Tokenizer

In [47]:
import torch

from transformers import BertForSequenceClassification
from transformers import Trainer,TrainingArguments

In [48]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


## Tokenization

In [49]:

train_encodings = tokenizer(
    train_texts,truncation=True,
    padding=True,
    max_length=128)

test_encodings = tokenizer(
    test_texts,
    truncation=True,
    padding=True,
    max_length=128)

## Create Torch Dataset

In [50]:
class TweetDataset(torch.utils.data.Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx])
for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
test_dataset = TweetDataset(test_encodings, test_labels)

# Load Pretrained BERT Model

In [51]:
model = BertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2)

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['bert.embeddings.LayerNorm.bias', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.word_embeddings.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.outp

# Training Configuration

In [53]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,fp16=True,
    save_total_limit=2,
    report_to="none")

# Trainer

In [54]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics)

# Train model

In [55]:
trainer.train()   ##take time 15-40minutes (GPU faster)

Step,Training Loss
500,0.258100
1000,0.094200
1500,0.081000
2000,0.071400
2500,0.063200
3000,0.052500
3500,0.045800
4000,0.047700
4500,0.042200
5000,0.036900


TrainOutput(global_step=5018, training_loss=0.07916955224020446, metrics={'train_runtime': 873.413, 'train_samples_per_second': 91.904, 'train_steps_per_second': 5.745, 'total_flos': 5279981103436800.0, 'train_loss': 0.07916955224020446, 'epoch': 2.0})

# Evaluate Model

In [56]:
trainer.evaluate()

{'eval_loss': 0.045871879905462265,
 'eval_runtime': 18.4739,
 'eval_samples_per_second': 543.146,
 'eval_steps_per_second': 33.994,
 'epoch': 2.0}

In [61]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Now when you evaluate:
trainer.evaluate()

{'eval_loss': 0.045871879905462265,
 'eval_model_preparation_time': 0.0045,
 'eval_accuracy': 0.990332868247957,
 'eval_runtime': 26.606,
 'eval_samples_per_second': 377.133,
 'eval_steps_per_second': 23.604}